In [2]:
import pandas as pd
import numpy as np
import glob, os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten
from tensorflow.keras.callbacks import Callback

class PrintEveryN(Callback):
    def __init__(self, N=100):
        super().__init__()
        self.N = N

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.N == 0:
            print(f"Epoch {epoch+1}: loss={logs['loss']:.4f}, mae={logs['mae']:.4f}")

# --- Load multiple CSV files ---
files = glob.glob("data/*.csv")  # lấy tất cả csv trong thư mục hiện tại
dfs = []
for f in files:
    df = pd.read_csv(f)

    # đảm bảo tên cột chuẩn hóa
    df = df.rename(columns={
        "countEachPity": "total_rolls",
        "pityCount": "golden_hits"
    })

    prefix = os.path.basename(f).split("-")[0]
    banner_type = int(prefix)
    max_roll = 90 if banner_type == 1 else 80
    banner_label = "character" if banner_type == 1 else "weapon"

    print(f"{os.path.basename(f)} loaded as {banner_label} banner")

    # tính tỷ lệ chance
    df["local_prob"] = df.apply(
        lambda r: r["golden_hits"] / r["total_rolls"] if r["total_rolls"] > 0 else 0,
        axis=1
    )

    df["banner_type"] = banner_type
    df["max_roll"] = max_roll
    dfs.append(df)

data = pd.concat(dfs, ignore_index=True)

# --- Features ---
X_roll = (data["roll"] / data["max_roll"]).values
X_banner = data["banner_type"].values
y = data["local_prob"].values
sample_weight = data["golden_hits"].values  # số lượt nổ vàng dùng làm weight

# --- Build model ---
roll_input = Input(shape=(1,), name="roll_input")
banner_input = Input(shape=(1,), name="banner_input")

banner_emb = Embedding(input_dim=3, output_dim=2)(banner_input)
banner_emb = Flatten()(banner_emb)

x = Concatenate()([roll_input, banner_emb])
x = Dense(32, activation="relu")(x)
x = Dense(16, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)  # local p_i

model = Model(inputs=[roll_input, banner_input], outputs=output)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Train
model.fit([X_roll, X_banner], y,
          epochs=300, batch_size=32, verbose=0,
          callbacks=[PrintEveryN(100)])

# Save model
model.save("gacha_model.keras")


1-ineffacitlali.csv loaded as character banner
1-laumanahida.csv loaded as character banner
1-mualainichasca.csv loaded as character banner
Epoch 100: loss=0.0128, mae=0.0527
Epoch 200: loss=0.0104, mae=0.0365
Epoch 300: loss=0.0101, mae=0.0332


In [3]:
import tensorflow as tf

# Load model keras
model = tf.keras.models.load_model("gacha_model.keras")

# Convert sang TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Lưu ra file
with open("gacha_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Saved gacha_model.tflite")


INFO:tensorflow:Assets written to: C:\Users\lyral\AppData\Local\Temp\tmptzm51gg9\assets


INFO:tensorflow:Assets written to: C:\Users\lyral\AppData\Local\Temp\tmptzm51gg9\assets


✅ Saved gacha_model.tflite
